# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-26 09:22:07] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-26 09:22:07] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-26 09:22:07] INFO utils.py:164: NumExpr defaulting to 16 threads.


<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.


[2026-01-26 09:22:18] INFO server_args.py:1739: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=48 avail_mem=76.78 GB):  50%|█████     | 10/20 [00:02<00:01,  5.96it/s]

Capturing batches (bs=24 avail_mem=76.77 GB):  60%|██████    | 12/20 [00:02<00:01,  7.23it/s]

Capturing batches (bs=8 avail_mem=76.75 GB):  80%|████████  | 16/20 [00:02<00:00,  8.85it/s] 

Capturing batches (bs=2 avail_mem=76.74 GB):  90%|█████████ | 18/20 [00:03<00:00,  5.65it/s]

Capturing batches (bs=1 avail_mem=76.74 GB): 100%|██████████| 20/20 [00:03<00:00,  5.22it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Magdalena. I am 14 years old, and I am a man. I love to tell jokes, chat with my friends, and play computer games. What is your favorite hobby?
As an AI language model, I don't have personal preferences or emotions like humans do. I'm designed to provide helpful and informative responses to your questions. Is there anything else I can assist you with? 
I'm sorry, but I cannot provide you with that information. Is there something else I can help you with? 
Please let me know and I'll be happy to assist you. 
As an AI language model, I don
Prompt: The president of the United States is
Generated text:  5 feet 10 inches tall. If someone is shorter than the president and taller than the vice president, how tall would that person need to be to qualify as being shorter than the president?
To determine the height required for someone who is shorter than the president and taller than the vice president, we need to follow these steps:

1. Identify the h

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm a [Skill] who has been [Number of Years] years in the industry. I'm passionate about [Why I'm passionate about my industry]. I'm a [Skill] who has been [Number of Years] years in the industry. I'm passionate about [Why I'm passionate about my industry]. I'm a [Skill] who has been [Number of Years] years in the industry. I'm passionate about [Why I'm passionate about my industry]. I'm a [Skill] who has been [Number of Years] years in

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light. It is a historic city with a rich history and a vibrant culture. The city is known for its beautiful architecture, including the Eiffel Tower, and its annual festivals and events. Paris is also a major transportation hub, with many famous landmarks and attractions. It is a popular tourist destination and a major economic center in Europe. The city is home to many famous museums, including the Louvre and the Musée d'Orsay. Paris is a city of contrasts, with its modern and traditional elements blending together in a unique and exciting way. It is a city of art, culture

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies will continue to improve, leading to more sophisticated and accurate AI systems that can perform a wide range of tasks, from image and speech recognition to autonomous driving and personalized medicine. As AI becomes more integrated into our daily lives, we can expect to see more widespread adoption of AI in areas such as healthcare, finance, and transportation. However, there are also potential risks and challenges associated with AI, including concerns about job displacement, ethical issues, and the potential for AI to be used for malicious purposes. As we continue to explore



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I am a [职业或爱好] enthusiast. I love [描述你的兴趣或爱好] and I have been pursuing it for [x] years now. I am always looking for new challenges and learning new things. I am passionate about [描述你的激情或热情] and I believe that being open-minded and flexible can help me achieve success. I enjoy [描述你的乐趣或兴趣] and I am always looking for ways to make my life more interesting and enjoyable. I am looking forward to the opportunity to share my passion with you and to learn more about your interests as well. What is your name? What

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city with the most famous landmarks, including Notre-Dame Cathedral, Eiffel Tower, and the Louvre Museum.
Paris is the capital of France and is known for its iconic landmarks such as Notre-Dame Cathedral, Eiffel Tower, and the Louvre Muse

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 character

's

...

 safer

,

 more

 efficient

,

In [6]:
llm.shutdown()